# Logistic Regression

In [7]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
# 0) prepare data

bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# change to torch
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape
y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)


[[1.288e+01 1.822e+01 8.445e+01 ... 1.096e-01 2.582e-01 8.893e-02]
 [1.113e+01 2.244e+01 7.149e+01 ... 6.413e-02 3.169e-01 8.032e-02]
 [1.263e+01 2.076e+01 8.215e+01 ... 1.105e-01 2.226e-01 8.486e-02]
 ...
 [1.247e+01 1.860e+01 8.109e+01 ... 1.015e-01 3.014e-01 8.750e-02]
 [1.822e+01 1.870e+01 1.203e+02 ... 1.325e-01 3.021e-01 7.987e-02]
 [1.272e+01 1.378e+01 8.178e+01 ... 6.343e-02 2.369e-01 6.922e-02]]
[[-0.36180827 -0.26521011 -0.31715702 ... -0.07967528 -0.52798733
   0.2506337 ]
 [-0.8632675   0.71560604 -0.85646012 ... -0.76980239  0.44312729
  -0.20987332]
 [-0.4334453   0.32513895 -0.41286667 ... -0.06601541 -1.1169427
   0.0329492 ]
 ...
 [-0.479293   -0.17689018 -0.45697634 ... -0.20261414  0.18670009
   0.17414996]
 [ 1.16835876 -0.15364809  1.17466524 ...  0.26789258  0.19828067
  -0.23394164]
 [-0.40765597 -1.29715887 -0.42826344 ... -0.78042674 -0.88036793
  -0.80355834]]


In [9]:
# 1) model

# f= wx + b -> sigmoid
class LogisticRegression1(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression1, self).__init__()
        self.Linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.Linear(x))
        return y_predicted


model = LogisticRegression1(n_features)


In [10]:
# 2) loss and optimizer
learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [11]:
# 3) training loop
num_epoch = 100

for epoch in range(num_epoch):
    # forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # backward pass
    loss.backward()

    # update
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch {epoch+1}: loss = {loss.item():.4f}')


epoch 10: loss = 0.5185
epoch 20: loss = 0.4485
epoch 30: loss = 0.3993
epoch 40: loss = 0.3625
epoch 50: loss = 0.3339
epoch 60: loss = 0.3108
epoch 70: loss = 0.2917
epoch 80: loss = 0.2757
epoch 90: loss = 0.2619
epoch 100: loss = 0.2500


In [12]:
# 4) predict and results
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'Accuracy: {acc:.4f}')


Accuracy: 0.8772
